In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [2]:
# Step 3: Import Groq LLM
from langchain_groq import ChatGroq

chatModel = ChatGroq(api_key="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", model="qwen/qwen3-32b")

## ***RunnablePassthrough***

In [4]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()
out = chain.invoke("Emon")
print(out)

Emon


In [5]:
print(type("Emon"))
print(type(out))

<class 'str'>
<class 'str'>


## ***RunnableLambda***

In [6]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)
print(chain.invoke("Emon"))

Emonovich


## ***RunnableParallel***

In [7]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

print(chain.invoke("Emon"))

{'operation_a': 'Emon', 'operation_b': 'Emonovich'}


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")
output_parser = StrOutputParser()

In [10]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | chatModel | output_parser

chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'<think>\nOkay, so I need to find a curious fact about Roman Abramovich. Let me start by recalling what I know about him. He\'s a Russian billionaire, right? I think he\'s involved in several businesses, like oil, metals, maybe sports teams. Oh, he owns Chelsea FC in the English Premier League. That\'s pretty well-known.\n\nNow, a curious fact. Maybe something about his early life, or some unusual investments, or perhaps a unique property he owns? I remember hearing that he owns a superyacht. Is there something unusual about it? Or maybe a private jet? Also, he\'s had some controversies, like sanctions in recent years due to his ties with Russia and Putin. But maybe that\'s too recent or not as curious.\n\nWait, there was something about a diamond collection. Or maybe his spending on art? Or perhaps a unique way he\'s used his wealth. Let me think. Oh, there was a time when he was involved in space tourism. He went to the International Space Station, right? That could be a curious fact

## ***RunnableParallel + FAISS Retrieval***

In [13]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."],
    embedding=embedding_model
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | chatModel
    | StrOutputParser()
)

retrieval_chain.invoke("What is dswithbappy?")


'<think>\nOkay, the user is asking, "What is dswithbappy?" and I need to answer based on the provided context. Let me look at the document they gave.\n\nThe document says: "dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English." So the key points here are that dswithbappy is a platform or maybe a blog, YouTube channel, or similar, that provides educational content in several tech fields. The areas mentioned are Data Science, Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, Natural Language Processing, and Python programming. The content is in English. \n\nI need to make sure the answer includes all those technical terms and that it\'s clear what the focus is. Also, the answer should be concise but cover all the topics listed. Let me check if there\'s any other info in the document, but it seems straightforward. The metadata is empty, so no other details there. \n\nSo the answer should state that dswi

## ***RunnableParallel + itemgetter + Multilingual Prompting***

In [14]:
from operator import itemgetter

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | chatModel
    | StrOutputParser()
)

chain.invoke({
    "question": "What is dswithbappy?",
    "language": "Pirate English"
})

'<think>\nOkay, let\'s tackle this. The user wants to know what dswithbappy is, and the answer needs to be in Pirate English. The context given is a document that says dswithbappy focuses on Data Science, AI, ML, DL, CV, NLP, Python programming, etc., in English.\n\nFirst, I need to translate the key terms into Pirate English. Let\'s see... "focuses on providing content" could be "be \'bout givin\' \'tention to". "Content on Data Science, AI, ML, DL, CV, NLP, Python programming" would be "datascience, AI, ML, DL, CV, NLP, Python programmin’ stuff". "In English" might be "in the King’s tongue" since pirates often refer to English as the King\'s or Queen\'s English. \n\nPutting it all together, the answer should start with "Arrr, dswithbappy be a site \'at be \'bout givin\' \'tention to..." followed by the list of topics. Need to make sure the pirate terms are consistent. Maybe add an "Arrr" at the beginning for authenticity. Also, check if any of the abbreviations like ML (Machine Learn